In [155]:
# Handling Inconsistent Data Entry 


In [179]:
#required libraries
import pandas as pd
import numpy as np
#!pip3 install fuzzywuzzy
import fuzzywuzzy
from fuzzywuzzy import process

C:\Users\pc\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [157]:
input_dataset=pd.read_csv("C:\\Users\\pc\\Documents\\edl\\data cleaning\\pakistan_intellectual_capital.csv")

In [158]:
input_dataset.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [159]:
# set seed for reproducibility
np.random.seed(0)

In [170]:
#checking inconsisteny in country column in the dataset
unique_countries=input_dataset['Country'].unique()
unique_countries.sort()
unique_countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

In [174]:
#From the list of countries it can be observed certain countries like Germany and New Zealand are repeated due to additional spaces in the value

#coverting all the values to lower case in the country column
input_dataset['Country'] = input_dataset['Country'].str.lower()
input_dataset['Country'] = input_dataset['Country'].str.strip()

In [175]:
unique_countries=input_dataset['Country'].unique()

In [176]:
unique_countries # south korea and southkorea are same thus using fuzzywuzzy package to identify the stings that match closest


array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'southkorea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', 'netherland',
       'greece', 'turkey', 'south korea', 'macau', 'singapore', 'spain',
       'japan', 'hongkong', 'saudi arabia', 'mauritius', 'scotland'],
      dtype=object)

In [ ]:
'''Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between
the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to 
"south korea".'''

In [181]:
matches = fuzzywuzzy.process.extract("south korea", unique_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('usofa', 25),
 ('macau', 25)]

In [198]:
def replace_value(dataframe,column,value,min_ratio=47):
    strings=dataframe[column].unique()
    matches = fuzzywuzzy.process.extract(value, unique_countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    # only get matches with a ratio > 90
    best_match = [match[0] for match in matches if match[1] >= min_ratio ]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = dataframe[column].isin(best_match)
    # replace all rows with close matches with the input matches 
    dataframe.loc[rows_with_matches, column] = value
    
    # let us know the function's done
    print("All done!")
    
    

In [199]:
replace_value(dataframe=input_dataset,column='Country',value='south korea')

All done!


In [200]:
unique_countries=input_dataset['Country'].unique()

In [201]:
unique_countries

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'south korea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', 'netherland',
       'greece', 'turkey', 'macau', 'singapore', 'spain', 'japan',
       'hongkong', 'saudi arabia', 'mauritius', 'scotland'], dtype=object)